In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '8'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(1, '../../src')

import torch
import evaluate
import mlflow
import numpy as np
import pandas as pd
from torch import nn
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

from TextProcessor import TextProcessor
from DatasetProcessor import DatasetProcessor


torch.cuda.empty_cache()

In [4]:
mlflow.set_tracking_uri("http://mlflow:5000")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

os.environ["MLFLOW_EXPERIMENT_NAME"] = "NER"
# os.environ["HF_MLFLOW_LOG_ARTIFACTS"] = "True"

# os.environ["MLFLOW_FLATTEN_PARAMS"] = "True"

# os.environ["MLFLOW_TRACKING_URI"] = tracking_uri

Current tracking uri: http://mlflow:5000


In [5]:
# raw_dataset = load_dataset("kosta-naumenko/medflex", split='train', download_mode='force_redownload', verification_mode='no_checks')
raw_train, raw_test = DatasetProcessor().make_dataset()
raw_train, raw_test

1209
Total train raws: 1934
Total test raws: 101


(Dataset({
     features: ['tokens', 'ner_tags'],
     num_rows: 1934
 }),
 Dataset({
     features: ['tokens', 'ner_tags'],
     num_rows: 101
 }))

In [6]:
model_name = "alexyalunin/RuBioRoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, 
        max_length=512, padding=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
train_dataset = raw_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_train.column_names,
)

test_dataset = raw_test.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_test.column_names,
)

Map:   0%|          | 0/1934 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [10]:
id = 4
input_ids, attention_mask, labels = list(train_dataset[id].values())
for i in range(len(input_ids)):
    if labels[i] > 0:
        if labels[i] == 1:
            print(" ")
        print(tokenizer.decode(input_ids[i]), end='')

 
 состояние тяжелое 
 Sat О 2 92% 
 пастозность стоп. 
 Sat О 2 92% 
 притупление в правой нижней доле 
 при аускультации - ослабленное дыхание, крепитация инспираторная в нижних отделах правого легкого 
 ЧДД - 22 в мин 
 Акцент II тона над Аортой 
 Артериальное давление 110/70 мм. рт. ст.

In [11]:
seqeval = evaluate.load("seqeval")
label_list = ['O', 'B', 'I']


def find_symtoms(result):
    text = result['text'].lower()
    symptoms = result['symptoms']
    for symptom in symptoms:
        text = text[:symptom[0]] + text[symptom[0]:symptom[1]].upper() + text[symptom[1]:]
    return text


def compute_metrics(p):
    global model, tokenizer, cur_texts, text_log_ids, processor
    model.eval()
    model.cpu()

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    for log_id in text_log_ids:

        if len(cur_texts[log_id]) == 0:
            result = processor.process_labeled_texts(
                raw_test[log_id]['tokens'],
                preds=torch.LongTensor([test_dataset[log_id]['labels']]),
                is_split=True
                )
            true_text = find_symtoms(result[0])
            cur_texts[log_id].append(true_text)

        result = processor.process_text(raw_test[log_id]['tokens'], model, is_split=True)
        cur_text = find_symtoms(result)
        cur_texts[log_id].append(cur_text)
        mlflow.log_text('\n'.join(cur_texts[log_id]), f'example_{log_id}.txt')

    model.train()
    model.to(torch.cuda.current_device())

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [12]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
}
label2id = {
    "O": 0,
    "B": 1,
    "I": 2,
}

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module
        self.adapter = nn.Sequential(
            nn.Linear(module.in_features, rank, bias=False),
            nn.Linear(rank, module.out_features, bias=False)
        )
        nn.init.kaiming_uniform_(self.adapter[0].weight, a=5 ** 0.5)
        nn.init.zeros_(self.adapter[1].weight)

        self.adapter.to(module.weight.device)

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (base module outputs + adapter outputs)
        return self.module(input) + self.adapter(input)

In [19]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    device_map={'': torch.cuda.current_device()},
    cache_dir='.cache',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
    )

for param in model.roberta.parameters():
    param.requires_grad = False

for name, module in model.roberta.named_modules():
    if 'RobertaSelfAttention' in repr(type(module)):
        module.query = LoRALayer(module.query, rank=128)
        module.key = LoRALayer(module.key, rank=128)
        module.value = LoRALayer(module.value, rank=128)

# model.classifier = nn.Sequential(
#     nn.Linear(1024, 512),
#     nn.Linear(512, 256),
#     nn.Linear(256, 3)
# )

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at alexyalunin/RuBioRoBERTa and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        # print(f'{item:>8}')
        pass
    print(f'________\n{sum(params):>8} - number of trainable parameters')

count_parameters(model)

________
18877443 - number of trainable parameters


In [21]:
mlflow.end_run()

text_log_ids = [0, 1, 2, 3, 4, 5]
cur_texts = [[] for _ in text_log_ids]
processor = TextProcessor()

cur_run_id = 1
num_train_epochs = 30

name = "head-1-LoRA-final"
run_name = f'{name}-{cur_run_id:02}'
output_dir = f'./logs/{run_name}'
batch_size = 16

training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_steps=1,
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to="mlflow",
    run_name=run_name,
    max_grad_norm=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
mlflow.end_run()

/home/knaumenko/.conda/envs/NER/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.582300,0.586745,0.342105,0.022414,0.042071,0.767419
2,0.465300,0.469087,0.016598,0.034483,0.022409,0.799183
3,0.499100,0.380645,0.057393,0.101724,0.073383,0.838799
4,0.343000,0.293227,0.097689,0.160345,0.121410,0.869334
5,0.256800,0.258485,0.182254,0.262069,0.214993,0.890629
6,0.264500,0.215356,0.263904,0.417241,0.323313,0.911052
7,0.114900,0.207478,0.281283,0.453448,0.347195,0.914780
8,0.238900,0.184646,0.339576,0.468966,0.393917,0.919578
9,0.118800,0.162573,0.397653,0.525862,0.452858,0.929730
10,0.065900,0.158566,0.418405,0.587931,0.488889,0.936075


In [22]:
path_to_model = f'../../models/{run_name}.pt'
torch.save(model, path_to_model)